In [39]:
pip install pandas transformers


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import pandas as pd

def count_words(text):
    words = text.split()
    return len(words)

def main(input_csv_file, output_csv_file):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(input_csv_file)

    # Calculate the number of words in the comment section and create a boolean mask
    df['word_count'] = df['comment'].apply(count_words)
    mask = df['word_count'] <= 100

    # Filter the DataFrame based on the mask
    df_ytb = df[mask]

    # Drop the 'word_count' column as it's no longer needed
    df_ytb = df_ytb.drop(columns=['word_count'])

    # Reset index to rearrange numbering
    df_ytb = df_ytb.reset_index(drop=True)

    # Write the filtered DataFrame to a new CSV file
    df_ytb.to_csv(output_csv_file, index=False)

    return df_ytb

if __name__ == "__main__":
    input_csv_file = "/content/drive/MyDrive/CSV FILES/ytb_Kingdom of Us.csv"
    output_csv_file = "output.csv"

    df_ytb = main(input_csv_file, output_csv_file)
    print(df_ytb)

    Unnamed: 0                                            comment  \
0            0  I went to school with the family. They all had...   
1            1  Einfach so eine emotionale Doku. Bitte schaut ...   
2            2                    Sad but also beautiful story...   
3            3  Thankyou for sharing your personal experiences...   
4            5  The kids are as bat shit crazy as the father! ...   
..         ...                                                ...   
57          63                                               Nice   
58          64                      It&#39;s too sad for me!!! 😞😭   
59          65                What&#39;s matter with you? netflix   
60          66                              Selfish piece of shit   
61          67                  Netflix has amazing documentaries   

               username  
0   TheDragonAndTheWolf  
1               leandra  
2               Aruja05  
3               Sid Man  
4        Rhonda Flowers  
..             

In [34]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

#df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/ytb_Kingdom of Us.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(62, 2)


In [35]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

[{'label': '2', 'score': 0.8682646751403809},
 {'label': '2', 'score': 0.4573045074939728},
 {'label': '2', 'score': 0.5188252329826355},
 {'label': '2', 'score': 0.6377865076065063},
 {'label': '2', 'score': 0.880934476852417},
 {'label': '2', 'score': 0.5377697348594666},
 {'label': '1', 'score': 0.5337073802947998},
 {'label': '2', 'score': 0.9481424689292908},
 {'label': '1', 'score': 0.39168021082878113},
 {'label': '2', 'score': 0.9098080992698669},
 {'label': '2', 'score': 0.7621977925300598},
 {'label': '1', 'score': 0.45443886518478394},
 {'label': '5', 'score': 0.46046411991119385},
 {'label': '1', 'score': 0.713595449924469},
 {'label': '1', 'score': 0.5514643788337708},
 {'label': '2', 'score': 0.6430814862251282},
 {'label': '2', 'score': 0.42740514874458313},
 {'label': '2', 'score': 0.5654460787773132},
 {'label': '2', 'score': 0.8552390336990356},
 {'label': '2', 'score': 0.5750610828399658},
 {'label': '2', 'score': 0.6238608956336975},
 {'label': '1', 'score': 0.60013

In [36]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,I went to school with the family. They all had...,2,0.868265
1,Einfach so eine emotionale Doku. Bitte schaut ...,2,0.457305
2,Sad but also beautiful story...,2,0.518825
3,Thankyou for sharing your personal experiences...,2,0.637787
4,The kids are as bat shit crazy as the father! ...,2,0.880934
...,...,...,...
57,Nice,2,0.534901
58,It&#39;s too sad for me!!! 😞😭,0,0.676616
59,What&#39;s matter with you? netflix,2,0.801937
60,Selfish piece of shit,2,0.669701


In [37]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         2           love
1         2           love
2         2           love
3         2           love
4         2           love
..      ...            ...
57        2           love
58        0        sadness
59        2           love
60        2           love
61        1            joy

[62 rows x 2 columns]


<ipython-input-37-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [38]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_Kingdom of Us_with_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,I went to school with the family. They all had...,2,0.868265,love
1,Einfach so eine emotionale Doku. Bitte schaut ...,2,0.457305,love
2,Sad but also beautiful story...,2,0.518825,love
3,Thankyou for sharing your personal experiences...,2,0.637787,love
4,The kids are as bat shit crazy as the father! ...,2,0.880934,love
...,...,...,...,...
57,Nice,2,0.534901,love
58,It&#39;s too sad for me!!! 😞😭,0,0.676616,sadness
59,What&#39;s matter with you? netflix,2,0.801937,love
60,Selfish piece of shit,2,0.669701,love
